In [25]:
import numpy as np
import pandas
from sklearn.preprocessing import OneHotEncoder

In [116]:
def movie_dict(arr):
    unique, counts = np.unique(arr, return_counts=True)
    k=len(counts)
    L=len(arr)
    movie_id_dict = {}

    for movie_id in arr:
        movie_id_dict[str(movie_id)] = -1
    for i,key in enumerate(movie_id_dict.keys()):
        movie_id_dict[key] = i
    return movie_id_dict
def get_movies_for_user(user,data,movie_mapping):
    data = data[data[:,0]==user]
    movies_user_seen=[]
    for movie in data[:,1]:
        movies_user_seen.append(movie_mapping[str(movie)])
    return movies_user_seen


def one_hot_encoding(data):
    #data = data.astype(int)
    L = data.shape[0]
    N = int(np.max(data[:,0]))
    unique, counts = np.unique(data[:,1], return_counts=True)
    k=len(counts)
    matrix_encoding_user = np.zeros((L,N))
    matrix_encoding_movie = np.zeros((L,k))
    matrix_encoding_other_movies_rated = np.zeros((L,k))
    matrix_encoding_last_movie_rated = np.zeros((L,k))
    movie_mapping = movie_dict(data[:,1])
    matrix_encoding_time = np.array(data[:,3])
    max_time=np.max(matrix_encoding_time)
    min_time=np.min(matrix_encoding_time)
    matrix_encoding_time = ((matrix_encoding_time-min_time)/max_time)

    for i,user in enumerate(data[:,0]):
        matrix_encoding_user[i,int(user)-1] = 1
    for i,movie in enumerate(data[:,1]):
        matrix_encoding_movie[i,movie_mapping[str(movie)]] = 1
    old_user=-1
    for i,user in enumerate(data[:,0]):
        if old_user!=user:
            user_movies = get_movies_for_user(user,data,movie_mapping)
        matrix_encoding_other_movies_rated[i,user_movies] = 1/len(user_movies)
    old_user=-1
    for i,user in enumerate(data[:,0]):
        if old_user==user:
            matrix_encoding_last_movie_rated[i,:] = matrix_encoding_movie[i-1,:]
        old_user=user
    
    
    full_matrix = np.concatenate((matrix_encoding_user,matrix_encoding_movie,matrix_encoding_other_movies_rated,matrix_encoding_time[:, None],matrix_encoding_last_movie_rated),axis=1)

    """ 
    def feed_forward(data,bias,bias_vector,v_matrix):
        N = int(np.max(data[:,0]))
        output = np.zeros(N)
        for i in range(0,N):
            comb_sum = 0
            first_part = bias + np.dot(bias_vector[i],data[i])
            for j in range(1,N):
                factor = np.dot(v_matrix[i],v_matrix[j])
                comb_sum += factor * data[i][j-1] * data[i][j] 
            output[i] =  first_part + comb_sum
        return output
    
    full_matrix = np.concatenate((matrix_encoding_user,matrix_encoding_movie),axis=1)
    print(full_matrix.shape)
    full_matrix = np.concatenate((full_matrix,matrix_encoding_other_movies_rated),axis=1)
    print(full_matrix.shape)
    print(matrix_encoding_time.shape)
    full_matrix = np.concatenate((full_matrix,matrix_encoding_time[:, None]),axis=1)
    print(full_matrix.shape)
    full_matrix = np.concatenate((full_matrix,matrix_encoding_last_movie_rated),axis=1)
    """
    return full_matrix



In [249]:
filename="ml-latest-small/ratings.csv"
data = pandas.read_csv(filename)
df = pandas.DataFrame(data)
df = df.iloc[:1000,:]
timestamp = df[["timestamp"]]
timestamp = (timestamp-timestamp.min())/(timestamp.max() - timestamp.min())
timestamp.reset_index(drop=True, inplace=True)
## One hot encode users and movies
encoder = OneHotEncoder(handle_unknown='ignore')
user_df = pandas.DataFrame(encoder.fit_transform(df[['userId']]).toarray())
user_df.reset_index(drop=True, inplace=True)
movie_df = pandas.DataFrame(encoder.fit_transform(df[['movieId']]).toarray())
movie_df.reset_index(drop=True, inplace=True)
##  Setting Other movies rated
user_id = df.iloc[:,:1]
user_id.reset_index(drop=True, inplace=True)
other_m_df = pandas.concat([user_id, movie_df], axis = 1)
user_rating = other_m_df.groupby('userId').sum()
n_reviews = other_m_df.groupby('userId').count()
user_rating = user_rating/n_reviews
other_m_df.set_index('userId', inplace=True)
other_m_df.update(user_rating)
other_m_df.reset_index(drop=True, inplace=True)
## Setting Last movie rated
# TODO: Repeated code - not good - fix
user_id = df.iloc[:,:1]
user_id.reset_index(drop=True, inplace=True)
last_movie_df = pandas.concat([user_id, movie_df], axis = 1)
last_movie_df = last_movie_df.groupby('userId').shift(1, fill_value = 0)
last_movie_df.reset_index(drop=True, inplace=True)
## Setting rating 
rating = df[["rating"]]
rating.reset_index(drop=True, inplace=True)
##
final = pandas.concat([user_df, movie_df, other_m_df, timestamp, last_movie_df, rating], axis=1)
print(final)







       0    1    2    3    4    5    6    0    1    2  ...  793  794  795  \
0    1.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  ...  0.0  0.0  0.0   
1    1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  ...  0.0  0.0  0.0   
2    1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
3    1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
4    1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
..   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
995  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
996  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
997  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
998  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
999  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   

     796  797  798  799  800  801  rating  
0    0.0  0.0  0.0  0.0  0.0  0